<a href="https://colab.research.google.com/github/EN10/BabyLlama/blob/main/Baby_Llama_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# training and inference code
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1517, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1517 (delta 4), reused 4 (delta 0), pack-reused 1507
Receiving objects: 100% (1517/1517), 1.23 MiB | 2.68 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [2]:
!mkdir ./llama2.c/data
# create vocab
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.model
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/raw/main/tok105/tok105.vocab

# use vocab to pretokenize training data
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.tar.gz
!cd ./llama2.c/data && tar -xf tok105.tar.gz

# compile run / inference executable
!cd llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok105.model

--2024-07-10 08:02:53--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.model
Resolving huggingface.co (huggingface.co)... 13.226.225.123, 13.226.225.74, 13.226.225.112, ...
Connecting to huggingface.co (huggingface.co)|13.226.225.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/65/c8/65c8615bfd6b6d94c8e62381ee6653578b439f71ff04b86aaefa9828dec8bb70/eed6827c1b4d876b982b9b628626dba444960a041857085e0d88c4a92aa8d3c6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tok105.model%3B+filename%3D%22tok105.model%22%3B&Expires=1720857773&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMDg1Nzc3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY1L2M4LzY1Yzg2MTViZmQ2YjZkOTRjOGU2MjM4MWVlNjY1MzU3OGI0MzlmNzFmZjA0Yjg2YWFlZmE5ODI4ZGVjOGJiNzAvZWVkNjgyN2MxYjRkODc2Yjk4MmI5YjYyODYyNmRiYTQ0NDk2MGEwNDE4NTcwODVlMGQ4OGM0YTkyYW

In [3]:
# train neural net
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=105 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=1 --max_iters=1 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 105
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 1
Overriding: max_iters = 1
tokens per iteration will be: 32,768
breaks down as: 4 grad accum steps * 1 processes * 32 batch size * 256 max seq len
Initializing a new model from scratch
num decayed parameter tensors: 36, with 935,040 parameters
num non-decayed parameter tensors: 11, with 1,408 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 4.7025, val loss 4.7022
0 | loss 4.7045 | lr 0.000000e+00 | 8279.61ms | mfu -100.00%
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 1: train loss 4.7025, val loss 4.7022
saving checkpoint to out
wrote 

In [4]:
# inference
!cd ./llama2.c && ./run out/model.bin -z data/tok105.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time  36D($,SNNdZe%%hdnY qKKoL4hD[–W2aFddkboo“;fx((ssss2â XAA,—’R>>SxQ%>>>ru   Q!ddd—b MX$>j/ eC'7F/WWxxWftMMMU(–c04T"bWr>snWe6–tZR"aMhyfqbhmrxwcB“H&&SOv™™“s2Z[%…[oaRHñ\"8B1E/77|–tWg<unk>a’ééqqok6\,é&âfxtttt(DKKK(oIP“UHHvvâ\4OO>HHHa"AU5fICPR
achieved tok/s: 838.926174
